### Notebook Description

This notebook is a user manual for backtesting custom strategies.

**How ​​to Use:**

1. **Mount Google Drive:** Connect to Google Drive.
2. **Upload Strategy File:** Upload your custom strategy `.py` file to the server.
3. **Upload Config:** Upload a YAML file containing your strategy settings.
4. **Run Backtest:** Run the backtest and view the results.

**Results:**
- Backtest logs and metrics
- HTML report (optional)

#### Original GitHub Link
- https://github.com/NeoMatrixAI/nb-runner
- https://github.com/NeoMatrixAI/strategy

# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings('ignore')

# Register Api Key

In [ ]:
USER_KEY = " "      #  Input your User key

print("USER_KEY:", USER_KEY if USER_KEY else "Not found")

# Health check

In [ ]:
import requests

root_url = f'https://zipline.fin.cloud.ainode.ai/'
headers={"API-KEY": USER_KEY}
requests.get(root_url, headers=headers).json()

# 1. Upload Module

In [ ]:
# Upload or Update module file
endpoint = 'upload/module'
url = root_url + endpoint

module_name = "momentum_utils" # example : sltp_utils, momentum_utils

base_path = "/content/drive/MyDrive/NeoMatrixAI/common"
file_path = os.path.join(base_path, module_name + ".py")

params = {'overwrite': True} # If overwrite is True, it will modify the existing file.

with open(file_path, "rb") as f:
    files = {"file": f}
    response = requests.post(url, headers=headers, params=params, files=files)

print("📂 Upload Response:", response.json())

# 2. Upload Strategy

In [30]:
import os

tradeType = "futures"
strategy_name = "multi_period_momentum"

In [ ]:
# Upload or Update strategy file
endpoint = 'upload/strategy'
url = root_url + endpoint

base_path = f"/content/drive/MyDrive/NeoMatrixAI/{tradeType}"
file_path = os.path.join(base_path, strategy_name, strategy_name + ".py")

params = {'tradeType': tradeType, 'overwrite': True} # If overwrite is True, it will modify the existing file.

with open(file_path, "rb") as f:
    files = {"file": f}
    response = requests.post(url, headers=headers, params=params, files=files)

print("📂 Upload Response:", response.json())

# 2. Upload configs

In [ ]:
# Upload YAML Config
endpoint = 'upload/config'
url = root_url + endpoint

# Workspace path
base_path = f"/content/drive/MyDrive/NeoMatrixAI/{tradeType}"
file_path = os.path.join(base_path, strategy_name, "config.yaml")

params = {'tradeType': tradeType, 'overwrite': True}

with open(file_path, "rb") as f:
    files = {"file": f}
    response = requests.post(url, headers=headers, params=params, files=files)

print("📂 Config Upload Response:", response.json())

# Run backtest

In [ ]:
import json
import requests
import os
from datetime import datetime

path = "/content/drive/MyDrive/NeoMatrixAI/backtest_report"

# Generate timestamp suffix (YYMMDDHHMM format)
timestamp = datetime.now().strftime("%y%m%d%H%M")

request_payload = {
    "config_name": strategy_name,
    "generate_pyfolio_report": True
}

endpoint = 'run/futures/backtest/v2'
url = root_url + endpoint

CLIENT_TIMEOUT_SECONDS = 1900*4

try:
    with requests.post(url, json=request_payload, headers=headers, stream=True, timeout=CLIENT_TIMEOUT_SECONDS) as response:
        response.raise_for_status()

        final_result_data = None

        for line in response.iter_lines():
            if not line:
                continue

            try:
                data = json.loads(line.decode('utf-8'))

                if data.get("status") == "running" and data.get("message") == "heartbeat":
                    print(f"[{datetime.now():%H:%M:%S}] Backtest is running...")
                else:
                    print("\n--- Received Final Result ---")
                    final_result_data = data

            except json.JSONDecodeError:
                print(f"WARN: Received non-JSON line: {line.decode('utf-8', errors='ignore')}")

        if final_result_data:
            status = final_result_data.get("status")

            if status == "success":
                print(f"\n--- Backtest execution successful (Status: {status}) ---")
                report_type = final_result_data.get('report_type')

                # Create directory if not exists
                if not os.path.exists(path):
                    os.makedirs(path)

                if report_type == 'html':
                    print("Report Type: HTML Report included.")
                    html_content = final_result_data.get('html_content')
                    logs = final_result_data.get('logs', 'No stderr logs.')

                    if html_content:
                        report_filename = os.path.join(path, f"backtest_report_{strategy_name}_{timestamp}.html")
                        with open(report_filename, "w", encoding="utf-8") as f:
                            f.write(html_content)
                        print(f"HTML report saved to '{report_filename}'.")

                    # Save log file
                    log_filename = os.path.join(path, f"backtest_log_{strategy_name}_{timestamp}.txt")
                    with open(log_filename, "w", encoding="utf-8") as f:
                        f.write(logs)
                    print(f"Execution logs saved to '{log_filename}'.")

                    # Print only last 50 lines to console (full log in file)
                    log_lines = logs.split('\n')
                    if len(log_lines) > 50:
                        print(f"\n--- Execution Logs (last 50 of {len(log_lines)} lines, full log in file) ---")
                        print('\n'.join(log_lines[-50:]))
                    else:
                        print("\n--- Execution Logs (stderr) ---")
                        print(logs)

                elif report_type == 'logs_only':
                    print(f"Report Type: Logs Only.")
                    print(f"Message: {final_result_data.get('message')}")
                    logs = final_result_data.get('logs', 'No stderr logs.')

                    # Save log file
                    log_filename = os.path.join(path, f"backtest_log_{strategy_name}_{timestamp}.txt")
                    with open(log_filename, "w", encoding="utf-8") as f:
                        f.write(logs)
                    print(f"Execution logs saved to '{log_filename}'.")

                    log_lines = logs.split('\n')
                    if len(log_lines) > 50:
                        print(f"\n--- Execution Logs (last 50 of {len(log_lines)} lines, full log in file) ---")
                        print('\n'.join(log_lines[-50:]))
                    else:
                        print("\n--- Execution Logs (stderr) ---")
                        print(logs)

            else:
                print(f"\n--- Backtest execution failed (Status: {status}) ---")
                print(json.dumps(final_result_data, indent=2, ensure_ascii=False))

                # Save logs even on failure
                logs = final_result_data.get('logs', '')
                if logs:
                    if not os.path.exists(path):
                        os.makedirs(path)
                    log_filename = os.path.join(path, f"backtest_log_{strategy_name}_{timestamp}_FAILED.txt")
                    with open(log_filename, "w", encoding="utf-8") as f:
                        f.write(logs)
                    print(f"Error logs saved to '{log_filename}'.")

        else:
            print("ERROR: Stream ended without receiving a final result.")

except requests.exceptions.Timeout:
    print(f"\n--- API call failed: Timeout after {CLIENT_TIMEOUT_SECONDS} seconds ---")
except requests.exceptions.HTTPError as e:
    print(f"\n--- API call failed: HTTP Error {e.response.status_code} ---")
    print(e.response.text)
except requests.exceptions.RequestException as e:
    print(f"\n--- API call failure: Request Error ---")
    print(f"Error: {e}")
except Exception as e:
    print(f"\n--- Unexpected error occurred ---")
    print(f"Error type: {type(e).__name__}, Content: {e}")

In [ ]:
final_result_data['metrics']